# Test trajectory planning in generated map

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from copy import deepcopy

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace
from planeslam.geometry.util import quat_to_R

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

drone_rotations = np.zeros((3,3,num_scans))
for i in range(num_scans):
    drone_rotations[:,:,i] = quat_to_R(drone_orientations[i])

In [ ]:
# Extract scans
num_scans = len(PC_data)
scans = []
avg_extraction_time = 0
for i in range(num_scans):
    start_time = time.time()
    scans.append(pc_to_scan(PC_data[i]))
    avg_extraction_time += time.time() - start_time
avg_extraction_time /= num_scans
print(avg_extraction_time)

### Open-loop registration (odometry)

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = quat_to_R(drone_orientations[0])
t_abs = drone_positions[0,:].copy()
traj_est = np.zeros((num_scans, 3))
traj_est[0] = t_abs
traj_Rs = np.zeros((3, 3, num_scans))
traj_Rs[:,:,0] = R_abs
avg_runtime = 0

R_hats = []
t_hats = []

loop_closure_dist_thresh = 10.0
index_spacing_thresh = 10

r_errors = []

for i in range(1, num_scans):
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1])
    #R_hat = np.array([[0,1,0],[1,0,0],[0,0,1]]) @ R_hat.T @ np.array([[0,1,0],[1,0,0],[0,0,1]])
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs
    traj_Rs[:,:,i] = R_abs

    R_hats.append(R_hat)
    t_hats.append(t_hat)

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

Form pose graph

In [ ]:
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.edge.edge_odometry import EdgeOdometry
from graphslam.pose.se3 import PoseSE3

from planeslam.geometry.util import R_to_quat

In [ ]:
vertices = []
edges = []

# Add first vertex
p = PoseSE3(traj_est[0], R_to_quat(traj_Rs[:,:,0]))
v = Vertex(0, p)
vertices.append(v)

# For each scan
for i in range(1, num_scans):
    # Add new vertex
    p = PoseSE3(traj_est[i], R_to_quat(traj_Rs[:,:,i]))
    v = Vertex(i, p)
    vertices.append(v)

    # Add odometry edge
    information = np.eye(6)
    estimate = PoseSE3(t_hats[i-1], R_to_quat(R_hats[i-1]))
    estimate.normalize()
    e = EdgeOdometry([i-1, i], information, estimate)
    edges.append(e)

In [ ]:
# Add loop closure edge
R_hat, t_hat = robust_GN_register(scans[59], scans[0])
information = np.eye(6)
estimate = PoseSE3(t_hat, R_to_quat(R_hat))
estimate.normalize()
e = EdgeOdometry([0, 59], information, estimate)
edges.append(e)

In [ ]:
g = Graph(edges, vertices)

In [ ]:
g.optimize()

In [ ]:
positions = np.zeros((num_scans, 3))
for i, v in enumerate(g._vertices):
    positions[i] = v.pose.position

In [ ]:
rotations = np.zeros((3, 3, num_scans))
for i, v in enumerate(g._vertices):
    rotations[:,:,i] = quat_to_R(v.pose.orientation)

In [ ]:
est_traj_trace = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(positions)))
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(drone_positions)))
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Generate Map

In [ ]:
# Initialize transformed scans
scans_transformed = []
for i in range(num_scans):
    scans_transformed.append(deepcopy(scans[i]))
    scans_transformed[i].transform(rotations[:,:,i], positions[i])

In [ ]:
merged = scans_transformed[0]

for s in scans_transformed[1:50]:
    merged = merged.merge(s, dist_thresh=7.5)
    merged.reduce_inside(p2p_dist_thresh=5)
    merged.remove_small_planes(area_thresh=1.0)
    merged.fuse_edges(vertex_merge_thresh=2.0)

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Trajectory planning

In [ ]:
from planeslam.geometry.box3d import Box3D
from planeslam.planning.LPM import LPM
from planeslam.planning.reach_planner import ReachPlanner
import planeslam.planning.params as params
from planeslam.planning.zonotope import Zonotope

In [ ]:
LPM_file = os.path.join(os.getcwd(),'..', 'data', 'planning_models', 'quadrotor_linear_planning_model.mat')

p_0 = np.array([-80.0, -60.0, 2.0])[:,None]
#r_body = Box3D(np.zeros(3), np.eye(3)/2)
r_body = Zonotope(np.zeros((3,1)), np.eye(3)/2)

planner = ReachPlanner(LPM_file, p_0, map=merged, r_body=r_body)
planner.p_goal = np.array([0.0, 0.0, 2.0])[:,None]

In [ ]:
import matplotlib.pyplot as plt
from planeslam.planning.reachability import compute_PRS, generate_collision_constraints, check_collision_constraints

lpm = LPM(LPM_file)

In [ ]:
v_0 = np.zeros((3,1))
a_0 = np.zeros((3,1))
FRS = compute_PRS(lpm, p_0, v_0, a_0)

In [ ]:
fig, ax = plt.subplots()

for zono in FRS[1:]:
    #print(zono)
    zono_2D = zono.index([0,1])
    zono_2D.plot(ax=ax, alpha=0.05)

In [ ]:
zono_map = []
for plane in merged.planes: 
    c = plane.center[:,None]
    G = np.diff(plane.vertices[:3], axis=0).T / 2
    zono_map.append(Zonotope(c, G))

In [ ]:
zono_map[0].index([0,1]).plot()

In [ ]:
FRS[-1]

In [ ]:
c = FRS[-1].c[params.OBS_DIM]
G = FRS[-1].G

# Find columns of G which are nonzero in k_dim ("k-sliceable")
# - this forms a linear map from the parameter space to workspace
k_col = list(set(np.nonzero(G[params.K_DIM,:])[1]))
k_slc_G = G[params.OBS_DIM][:,k_col]

# "non-k-sliceable" generators - have constant contribution regardless of chosen trajectory parameter
k_no_slc_G = G[params.OBS_DIM]
k_no_slc_G = np.delete(k_no_slc_G, k_col, axis=1)

In [ ]:
# Get current obstacle
obs = zono_map[0].Z

# Obstacle is "buffered" by non-k-sliceable part of FRS
buff_obs_c = obs[:,0][:,None] - c
buff_obs_G = np.hstack((obs[:,1:], k_no_slc_G))
#buff_obs_G = remove_zero_columns(buff_obs_G)
buff_obs = Zonotope(buff_obs_c, buff_obs_G)

A_obs, b_obs = buff_obs.halfspace()

In [ ]:
obs

In [ ]:
c = np.zeros((3,1))
G = np.array([[1, 0], [0, 1], [0, 0]])
Z = np.hstack((c,G))

In [ ]:
np.linalg.svd(G)

In [ ]:
S,_,V = np.linalg.svd(G)

In [ ]:
import scipy


scipy.linalg.svd(G)

In [ ]:
V

In [ ]:
A_con, b_con = generate_collision_constraints(FRS, [zono_map[0]])

In [ ]:
v_peak = np.array([1, 1, -1.5])[:,None]
k = np.hstack((v_0, a_0, v_peak))
lpm.compute_positions(k)

In [ ]:
lambdas = v_peak / params.V_MAX
lambdas

In [ ]:
print(A_con[29], b_con[29])

In [ ]:
A_con[29] @ lambdas - b_con[29]

In [ ]:
check_collision_constraints(A_con, b_con, v_peak)

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p = p_0
v = np.zeros((3,1))
a = np.zeros((3,1))
# State history
P = []
planning_times = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan((p,v,a))
        planning_times.append(time.time() - start_t)
        traj_idx = 0

        if trajectory is None:
            print("Replanning failed, stopping")
            break

    # Controller
    # Pop off next setpoint
    p = trajectory[0][:,traj_idx][:,None]
    v = trajectory[1][:,traj_idx][:,None]
    a = trajectory[2][:,traj_idx][:,None]
    P.append(p)
    traj_idx += 1

    if np.linalg.norm(p - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

P = np.hstack(P)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
planning_times

In [ ]:
# Plot trajectory
fig = go.Figure(data=merged.plot_trace()+[go.Scatter3d(x=P[0,:], y=P[1,:], z=P[2,:], marker=dict(size=2))])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
lpm = LPM(LPM_file)

k = np.array([[0, 0, 5],
              [0, 0, 5],
              [0, 0, 5]])
positions = lpm.compute_positions(k)

In [ ]:
v_0 = np.array([1,2,3])
a_0 = np.array([2,3,4])
k_0 = np.vstack((v_0, a_0))

In [ ]:
from planeslam.planning.reachability import compute_PRS

PRS = compute_PRS(lpm, v_0, a_0)

In [ ]:
z = Zonotope(np.zeros((3,1)), np.eye(3))
z.contains(np.array([0,0,1.1])[:,None])

Test generate collision constraints

In [ ]:
from planeslam.general import remove_zero_columns

In [ ]:
k_dim = np.array([6, 9])
obs_dim = np.array([0, 1])

In [ ]:
obs = np.array([[3, 1, 0], [-3, 0, 1]])

In [ ]:
c = np.array([5.43, 5.0])[:,None]
G = np.array([[0, 0, 0.0807, -0.0462, 0.0348, -0.0846], 
              [0, 0, 0.0462, 0.0807, -0.0846, -0.0348],
              [0, 0, -0.0757, -0.0433, 0.0371, -0.0902],
              [0, 0, -0.0433, -0.0757, -0.0902, -0.0371],
              [0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [3, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [0, 3, 0, 0, 0, 0]])

In [ ]:
# Find columns of G which are nonzero in k_dim ("k-sliceable")
k_col = list(set(np.nonzero(G[k_dim,:])[1]))

In [ ]:
k_slc_G = G[obs_dim][:,k_col]

In [ ]:
# "non-k-sliceable" generators - have constant contribution regardless of chosen trajectory parameter
k_no_slc_G = G[obs_dim]
k_no_slc_G = np.delete(k_no_slc_G, k_col, axis=1)

In [ ]:
# Obstacle is "buffered" by non-k-sliceable part of FRS
buff_obs_c = obs[:,0][:,None] - c
buff_obs_G = np.hstack((obs[:,1:], k_no_slc_G))
buff_obs_G = remove_zero_columns(buff_obs_G)
buff_obs = np.hstack((buff_obs_c, buff_obs_G))

Test zonotope to halfspace conversion

In [ ]:
buff_obs = Zonotope(buff_obs_c, buff_obs_G)

In [ ]:
A_obs, b_obs = buff_obs.halfspace()

In [ ]:
a = np.array([0,0,1])[:,None]
np.kron(np.eye(3),a)

In [ ]:
Z = buff_obstacle
c = Z[:,0][:,None]
G = Z[:,1:]

(dim, n_gen) = G.shape

In [ ]:
#if dim > 1:

# Build C matrices
#if dim == 2:
C = G
C = np.vstack((-C[1,:], C[0,:]))# get perpendicular vector

In [ ]:
# if dim == 3:
comb = list(itertools.combinations(np.arange(n_gen), dim-1))

In [ ]:
# Normalize normal vectors
C = np.divide(C, np.linalg.norm(C, axis=0)).T

In [ ]:
# Compute dPos, dNeg
d = C @ c

In [ ]:
A = np.vstack((C, -C))
b = np.vstack((d + deltaD, -d + deltaD))

In [ ]:
b

In [ ]:
zero_idx = np.argwhere(np.all(A[...,:]==0, axis=0))

In [ ]:
np.delete(A, zero_idx, axis=1)